<a href="https://colab.research.google.com/github/Mrinal-Shankar/Music-Generation/blob/master/Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,TimeDistributed,Activation
from music21 import *

In [0]:
directory="/content/drive/My Drive/Music Generation"
filename="/content/drive/My Drive/Music Generation/Data_Tunes.txt"
SEQ_LENGTH=64
BATCH_SIZE=16
limit=200

In [0]:
def preprocess(data):
  list1=list(data)
  list2=['\n','\n','\n']
  ignore=['X','T','M','S','K','P']
  i=0
  #to remove Part1:
  while(i<len(list1)):
    if(((list1[i] in ignore) and (list1[i+1]==":"))or list1[i]=='%' ):
      del list2[-1]
      while(list1[i]!='\n'):
        i=i+1
    list2.append(list1[i])
    i=i+1
  i=0
  #to append 'Z'(start token)
  preprocess_data=[]
  while(i<len(list2)):
    if(list2[i]=='\n'and list2[i+1]=='\n' and list2[i+2]=='\n'):
      preprocess_data.append('Z')
      i=i+3
    else:
      preprocess_data.append(list2[i])
      i=i+1
  return preprocess_data

In [0]:
def read_data(preprocess_data):
  char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(preprocess_data))))}

    
  #with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        #json.dump(char_to_index, f)
        
  index_to_char = {i: ch for (ch, i) in char_to_index.items()}
  num_unique_chars = len(char_to_index)
  all_characters_as_indices = np.asarray([char_to_index[c] for c in preprocess_data], dtype = np.int32)
  return all_characters_as_indices,num_unique_chars,char_to_index,index_to_char



In [0]:
file=open(filename,mode='r')
data=file.read()
file.close()

preprocessed_data=preprocess(data)

In [0]:
 all_characters_as_indices,num_unique_characters,char_to_index,index_to_char=read_data(preprocessed_data)

In [7]:
print(num_unique_characters)

59


In [8]:
len(all_characters_as_indices)

116963

In [9]:
print(char_to_index)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, "'": 5, '(': 6, ')': 7, '+': 8, ',': 9, '-': 10, '/': 11, '1': 12, '2': 13, '3': 14, '4': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, '=': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'L': 30, 'R': 31, 'V': 32, 'Z': 33, '[': 34, '\\': 35, ']': 36, '^': 37, '_': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'i': 46, 'l': 47, 'm': 48, 'n': 49, 'o': 50, 'p': 51, 'r': 52, 's': 53, 't': 54, 'u': 55, 'z': 56, '|': 57, '~': 58}


In [0]:
def input_output(all_chars_as_indices,num_unique_chars):

  total_length=all_chars_as_indices.shape[0]
  num_examples=int(total_length/SEQ_LENGTH)
  X=np.zeros((num_examples,SEQ_LENGTH))
  Y=np.zeros((num_examples,SEQ_LENGTH,num_unique_chars))
  for i in range(num_examples):
    for j in range(SEQ_LENGTH):
      X[i,j]=all_chars_as_indices[i*SEQ_LENGTH+j]
      Y[i,j,all_chars_as_indices[i*SEQ_LENGTH+j+1]]=1
  return X,Y

   

In [0]:
x,y=input_output(all_characters_as_indices,num_unique_characters)

In [0]:
def build_model(seq_length,num_unique_chars):
  model = tf.keras.Sequential()
    
  model.add(Embedding(input_dim = num_unique_chars, output_dim = 512, input_shape = (seq_length,))) 
    
  model.add(LSTM(256, return_sequences = True))
  model.add(Dropout(0.2))
    
  model.add(LSTM(256, return_sequences = True))
  model.add(Dropout(0.2))
  model.add(LSTM(256, return_sequences = True))
  model.add(Dropout(0.2))
    
    
  model.add(TimeDistributed(Dense(num_unique_chars)))

  model.add(Activation("softmax"))
    
  return model

In [0]:
model=build_model(SEQ_LENGTH,num_unique_characters)

In [14]:
model.compile(loss='categorical_crossentropy',
               optimizer='adam',
                metrics="accuracy")
model.summary()
checkpoint=ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5',monitor='loss',save_best_only=True,save_weights=True,save_every='10')
model.fit(x,y,batch_size=BATCH_SIZE,epochs=80,callbacks=[checkpoint])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 512)           30208     
_________________________________________________________________
lstm (LSTM)                  (None, 64, 256)           787456    
_________________________________________________________________
dropout (Dropout)            (None, 64, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64, 256)           525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 256)           0

In [0]:
def make_model(num_unique_chars):
  model = tf.keras.Sequential()
    
  model.add(Embedding(input_dim = num_unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
# stateful: If True, the last state for each sample at index i in a batch will be used 
# as initial state for the sample of index i in the following batch.
  model.add(LSTM(256, return_sequences = True, stateful = True))
  model.add(Dropout(0.2))
    
  model.add(LSTM(256, return_sequences = True, stateful = True))
  model.add(Dropout(0.2))
    
  model.add(LSTM(256,return_sequences=True, stateful = True)) 
  model.add(Dropout(0.2))
    
  model.add((Dense(num_unique_chars)))
  model.add(Activation("softmax"))
    
  return model

In [0]:
def generate_sequence(gen_seq_length,num_unique_chars,index_to_char):
   
    model = make_model(num_unique_chars)
    model.load_weights("/content/weights.80.hdf5")
     
    sequence_index = [char_to_index['Z']]

    for _ in range(gen_seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = np.squeeze(model.predict_on_batch(batch))
        sample = np.random.choice(range(num_unique_chars), size = 1, p = predicted_probs) 
      
        sequence_index.append(sample[0])
    
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    seq='M:6/8\n'+str(seq)
    return seq

In [0]:
music=generate_sequence(limit,num_unique_characters,index_to_char)

In [18]:
music

'M:6/8\nZA|"D"FGA "G"B2A|"D"def "G"gab|"D"fed "A7"gec|"D"d3 d2:|ZA|"D"ded dcd|"G"BGG dcB|"D"AGF "A7"E2A|"D"FAD FGA|"D"FGA A2d|"A"cec "D"d3:|Z"C"cBA GEG|"D"ABc dAc|"G"BGG G2d|\n"Am"eAA c2d|"Am"eAA eAA|"G"BAB GBd'

In [0]:
def midi_converter(music):
   c = converter.subConverters.ConverterABC()
   c.registerOutputExtensions = ("midi", )
   c.parseData(music)
   s = c.stream
   s.write('midi', fp='demos.mid')

In [0]:
midi_converter(music)